In [1]:
"""
 TODO: put there apis for:
  * Configuration
  * Anomalies
  * Raw CSV Upload
"""

import cloudberry.api as cb
cb_port = 9000
cb_config = cb.CloudberryConfig(f'http://localhost:{cb_port}')

In [2]:
# Utils
import math
import datetime
get_time = lambda: math.trunc(datetime.datetime.utcnow().timestamp())

In [3]:
# Raw data API
data_api = cb.Data(cb_config)

points = [
    cb.DataPoint(time=get_time(), fields={'version': 0.1}, tags={'origin': 'jupyter_notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=get_time() + 1, fields={'version': 0.2}, tags={'origin': 'jupyter_notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=get_time() + 2, fields={'version': 0.3}, tags={'origin': 'notebook', 'type': 'api_demo'}),
    cb.DataPoint(time=get_time() + 3, fields={'version': 0.4}, tags={'origin': 'notebook', 'type': 'api_demo'}),
]

In [4]:
# Save raw data points

data_api.save_data(points)

True

In [5]:
# Query raw data points - by tags values

filters = cb.DataFilters(tags={'origin': 'notebook'}, tags_presence=None, fields=None)
series = data_api.get_data(filters)

series.as_data_frame

,_time,_measurement,origin,type,version,series_name
0,1.604009e+09,cloudberry-default,notebook,api_demo,0.3,raw_data
1,1.604009e+09,cloudberry-default,notebook,api_demo,0.4,raw_data


In [6]:
# Query raw data points - by tags presence, regardless of their values

filters = cb.DataFilters(tags_presence=['origin'])
series = data_api.get_data(filters)

series.as_data_frame

,_time,_measurement,origin,type,version,series_name
0,1.604009e+09,cloudberry-default,jupyter_notebook,api_demo,0.1,raw_data
1,1.604009e+09,cloudberry-default,jupyter_notebook,api_demo,0.2,raw_data
2,1.604009e+09,cloudberry-default,notebook,api_demo,0.3,raw_data
3,1.604009e+09,cloudberry-default,notebook,api_demo,0.4,raw_data


In [8]:
# Buckets (InfluxDB-specific) API

buckets_api = cb.Buckets(cb_config)

In [9]:
# Get present buckets

buckets_api.get_buckets_names()

['cloudberry_logs', '_monitoring', '_tasks', 'cloudberry_metrics']

In [10]:
# Create new bucket

bucket_name = 'wiaderko_danych_0'
buckets_api.create_bucket(bucket_name)

buckets_api.get_buckets_names() # verify

['cloudberry_logs',
 '_monitoring',
 'wiaderko_danych_0',
 '_tasks',
 'cloudberry_metrics']

In [11]:
# Delete new bucket

buckets_api.delete_bucket(bucket_name)

buckets_api.get_buckets_names() # verify

['cloudberry_logs', '_monitoring', '_tasks', 'cloudberry_metrics']

In [12]:
# Raw Flux Query (InfluxDB-specific) API

query_api = cb.Query(cb_config)
query_flux = """
from(bucket:"cloudberry_logs")
|> range(start: 0)
|> filter(fn: (r) => r._field == "version")
|> keep(columns: ["_value", "_time"])
"""

query_series = query_api.query_series(query_flux)
query_series.as_data_frame

,result,table,_time,_value,series_name
0,_result,0,1.604009e+09,0.1,"\nfrom(bucket:""cloudberry_logs"")\n|> range(sta..."
1,_result,0,1.604009e+09,0.2,"\nfrom(bucket:""cloudberry_logs"")\n|> range(sta..."
2,_result,0,1.604009e+09,0.3,"\nfrom(bucket:""cloudberry_logs"")\n|> range(sta..."
3,_result,0,1.604009e+09,0.4,"\nfrom(bucket:""cloudberry_logs"")\n|> range(sta..."


In [13]:
# Delete created raw data points

data_api.delete_data(filters)

True

In [18]:
# Create metadata necessary for CSV upload - Experiment & Configuration

meta_api = cb.Metadata(cb_config)
experiment_api = meta_api.experiment_api()
configuration_api = meta_api.experiment_configuration_api()

csv_experiment_name = 'Example CSV Experiment'
csv_configuration_name = 'Example CSV Configuration'

# optionally pass parameters to save experiment with
csv_experiment = experiment_api.find_or_create(csv_experiment_name, parameters={'importance': 'high'})
csv_configuration = configuration_api.find_or_create(csv_experiment, csv_configuration_name, parameters={'version': 1})

In [21]:
# CSV-like Data Upload API

csv_api = cb.CsvFileUploader(cb_config)

csv_file_path = './data/emas_1.csv'
csv_details = cb.CsvUploadDetails(
    tags_names=['WORKPLACE_ID'],
    configuration=csv_configuration,
    computation=None # None for new one, otherwise append data to existing computation 
)

csv_computation = csv_api.upload_file(
    csv_file_path,
    csv_experiment_name,
    csv_details
)

In [22]:
# Verify uploaded CSV data

filters = cb.DataFilters(tags_presence=['WORKPLACE_ID'])
series = data_api.get_data(filters)
series.as_data_frame

,_time,WORKPLACE_ID,_measurement,computationId,POPULATION_SIZE,AVERAGE_FITNESS,ENERGY_SUM,STEP_NUMBER,series_name
0,5.003171e+06,0,cloudberry-default,5f9b4f796b9caa425950693c,98.0,4.598373,2460.512629,5079.0,raw_data
1,1.000203e+07,0,cloudberry-default,5f9b4f796b9caa425950693c,98.0,4.595291,2454.008962,10478.0,raw_data
2,1.500043e+07,0,cloudberry-default,5f9b4f796b9caa425950693c,99.0,4.607681,2529.148288,15843.0,raw_data
3,2.000397e+07,0,cloudberry-default,5f9b4f796b9caa425950693c,76.0,4.608023,1974.607684,20914.0,raw_data
4,5.003171e+06,0,cloudberry-default,5f9b4f816b9caa425950693e,98.0,4.598373,2460.512629,5079.0,raw_data
5,1.000203e+07,0,cloudberry-default,5f9b4f816b9caa425950693e,98.0,4.595291,2454.008962,10478.0,raw_data
6,1.500043e+07,0,cloudberry-default,5f9b4f816b9caa425950693e,99.0,4.607681,2529.148288,15843.0,raw_data
7,2.000397e+07,0,cloudberry-default,5f9b4f816b9caa425950693e,76.0,4.608023,1974.607684,20914.0,raw_data
8,5.003171e+06,0,cloudberry-default,5f9b4f896b9caa4259506940,98.0,4.598373,2460.512629,5079.0,raw_data
9,1.000203e+07,0,cloudberry-default,5f9b4f896b9caa4259506940,98.0,4.595291,2454.008962,10478.0,raw_data


In [ ]:
# # Anomalies Reports API

# # todo: fix NPEs

# anomalies_api = cb.Anomalies(cb_config)
# anomalies_api.get_report(field_name='AVERAGE_FITNESS', computation=csv_computation)

In [ ]:
# Cleanup
data_api.delete_data(filters)

In [ ]:
# Configuration Properties API

# todo: check if bucket is actually created

properties_api = cb.ApiConfiguration(cb_config)

property_key = cb.ApiPropertiesIndex.OVERRIDDEN_DEFAULT_BUCKET_NAME

properties_api.set_property(property_key, "cloudberry_logs_secondary")
property_value = properties_api.get_property(property_key)
print(property_value)

properties_api.delete_property(property_key)

property_value = properties_api.get_property(property_key)
print(property_value)

In [ ]:
# # #49 - append metadata ids to DataSeries

# # todo: all

# computation = cb_meta_experiment_computation.find_all()[1]
# computation_series = cb_analytics.compare_computations([computation], 'AVERAGE_FITNESS')[0]

# computation_series.meta_ids = cb_meta.get_meta_ids(computation)
# computation_series.as_data_frame


In [27]:
# # #49 - append metadata ids to DataSeries

# # todo: all

# computation = cb_meta_experiment_computation.find_all()[1]
# computation_series = cb_analytics.compare_computations([computation], 'AVERAGE_FITNESS')[0]

# computation_series.meta_ids = cb_meta.get_meta_ids(computation)
# computation_series.as_data_frame


TypeError: string indices must be integers

In [ ]:
# Cleanup
data_api.delete_data(filters)

In [24]:
# Configuration Properties API

# todo: check if bucket is actually created

properties_api = cb.ApiConfiguration(cb_config)

property_key = cb.ApiPropertiesIndex.OVERRIDDEN_DEFAULT_BUCKET_NAME

properties_api.set_property(property_key, "cloudberry_logs_secondary")
property_value = properties_api.get_property(property_key)
print(property_value)

properties_api.delete_property(property_key)

property_value = properties_api.get_property(property_key)
print(property_value)

"cloudberry_logs_secondary"



In [ ]:
# # #49 - append metadata ids to DataSeries

# # todo: all

# computation = cb_meta_experiment_computation.find_all()[1]
# computation_series = cb_analytics.compare_computations([computation], 'AVERAGE_FITNESS')[0]

# computation_series.meta_ids = cb_meta.get_meta_ids(computation)
# computation_series.as_data_frame
